In [ ]:
import sys
import os

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import datetime
import copy

# Local files
__root__ = '../../'
__pydirs__ = ['py', 'settings']
module_path = os.path.abspath(os.path.join(__root__[:-1]))
if module_path not in sys.path:
    for pydir in __pydirs__:
        sys.path.append(f"{module_path}/{pydir}/")

import T23_setup

import log_ret

import misc_functions as misc_fns

In [ ]:
_setup_T23 = T23_setup.setup_T23(__C__='Cm3', __fit__=True, __vares__=True,
                                    set_cv=False, set_tt=True,
                                    all_end=pd.Timestamp('2023-09-16')
                                    )

In [ ]:
for x in _setup_T23['a_q']:
    print(x)

In [ ]:
for x in _setup_T23['tt_a_u']:
    print(x)

In [ ]:
B20 = log_ret.log_ret(dir_base=__root__, ticks=_setup_T23['ticks'])
B20.set_tk(**misc_fns.dict_key_rm(_setup_T23, keys_rm='ticks'))

In [ ]:
for tick in B20.tk:
    print(B20.tk[tick].pd.lr['p']['all'].index[0],
            B20.tk[tick].pd.lr['p']['train'].index[-1], 
            B20.tk[tick].pd.lr['p']['forecast'][0][0].index[0], 
            B20.tk[tick].pd.lr['p']['all'].index[-1])

In [ ]:
for fix_a_lambda in [True, False]:
    for i in range(0, _setup_T23['tt_a_u'].size):
        B20.set_tt_hb_mp_tick(write_tick=True, 
                            a_u_rng=[i], tt_read_params=True, tt_hb_read_params=False, tt_fit_if_need=False, tt_fix_a_lambda=fix_a_lambda, append_fit_tt_w=True, set_eval=True, append_fit_tt_hb_w=True, append_vrs_tt_hb_w=True)